## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [8]:
import os
import os.path
import glob
import numpy as np
import scipy as sp
import scipy.io
import scipy.signal
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import pickle

def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def PreprocessTestData(data_fl, ref_fl): 
    
    """
    This function preprocess the data loaded from the Troika file
    """
    
    fs=125
    win_len = 8
    win_shift = 2
    
    # load the troika data
    sig = LoadTroikaDataFile(data_fl)
    ref = scipy.io.loadmat(ref_fl)["BPM0"]
    ref = np.array([x[0] for x in ref])
    subject_name = os.path.basename(data_fl).split('.')[0]  
    # find the star and end index of the signal
    
    start_indxs, end_indxs = get_indxs(sig.shape[1], len(ref), fs, win_len,win_shift)
    targets, features, sigs, subs = [], [], [], []
    for i, s in enumerate(start_indxs):
        start_i =  start_indxs[i]
        end_i = end_indxs[i]

        ppg = sig[0, start_i:end_i]            
        accx = sig[1, start_i:end_i]
        accy = sig[2, start_i:end_i]
        accz = sig[3, start_i:end_i]

        # band pass the channels
        ppg = BandpassFilter(ppg)
        accx = BandpassFilter(accx)
        accy = BandpassFilter(accy)
        accz = BandpassFilter(accz)
 
        # creates the features
        feature, ppg, accx, accy, accz = FeatureExtraction(ppg, accx, accy, accz)

        sigs.append([ppg, accx, accy, accz])
        targets.append(ref[i])
        features.append(feature)
        subs.append(subject_name)
        
    return (np.array(targets), np.array(features), sigs, subs)

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def FeatureExtraction(ppg, accx, accy, accz):
    """ 
    This function creates features 
    """

    fs = 125
    n = len(ppg) * 4
    # applying fast Fourier transform
    freqs = np.fft.rfftfreq(n, 1/fs)
    fft = np.abs(np.fft.rfft(ppg,n))
    fft[freqs <= 40/60.0] = 0.0
    fft[freqs >= 240/60.0] = 0.0
    
    ## calculating L2 norm
    acc_mag = np.sqrt(accx**2 + accy**2 + accz**2)
    acc_fft = np.abs(np.fft.rfft(acc_mag, n))
    acc_fft[freqs <= 40/60.0] = 0.0
    acc_fft[freqs >= 240/60.0] = 0.0
    
    ppg_feature = freqs[np.argmax(fft)]
    acc_feature = freqs[np.argmax(acc_fft)]
    
    return (np.array([ppg_feature, acc_feature]), ppg, accx, accy, accz)

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    global reg
    reg = TrainRegressor()
    
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def BandpassFilter(signal, fs = 125):  
    """
    This is a function to bandpass the signals between 40 and 240
    """
    b, a = scipy.signal.butter(3, (40/60.0, 240/60.0), btype='bandpass', fs=fs)
    return scipy.signal.filtfilt(b, a, signal)

def get_indxs(sig_len, ref_len, fs=125, win_len_s=10, win_shift_s=2):
    """
    Find start and end index to iterate over a set of signals
    """
    if ref_len < sig_len:
        n = ref_len
    else:
        n = sig_len
    
    start_indxs = (np.cumsum(np.ones(n) * fs * win_shift_s) - fs * win_shift_s).astype(int)
    end_indxs = start_indxs + win_len_s * fs
    return (start_indxs, end_indxs)

def TrainRegressor():
    """
    This function trains a Random Forest Regressor
    """
    
    fs=125
    win_len = 8
    win_shift = 2
    
    # load the data file
    data_fls, ref_fls = LoadTroikaDataset()
    targets, features, sigs, subs = [], [], [], []
    for data_fl, ref_fl in (zip(data_fls, ref_fls)):
        
        # load the signal
        sig = LoadTroikaDataFile(data_fl)
        ref = scipy.io.loadmat(ref_fl)["BPM0"]
        ref = np.array([x[0] for x in ref])
        subject_name = os.path.basename(data_fl).split('.')[0]        
        start_indxs, end_indxs = get_indxs(sig.shape[1], len(ref), fs, win_len,win_shift)
        for i, s in enumerate(start_indxs):
            start_i =  start_indxs[i]
            end_i = end_indxs[i]

            ppg = sig[0, start_i:end_i]            
            accx = sig[1, start_i:end_i]
            accy = sig[2, start_i:end_i]
            accz = sig[3, start_i:end_i]
            
            #band pass the channels
            ppg = BandpassFilter(ppg)
            accx = BandpassFilter(accx)
            accy = BandpassFilter(accy)
            accz = BandpassFilter(accz)
            
            # creates the features
            feature, ppg, accx, accy, accz = FeatureExtraction(ppg, accx, accy, accz)

            sigs.append([ppg, accx, accy, accz])
            targets.append(ref[i])
            features.append(feature)
            subs.append(subject_name)
            
    targets = np.array(targets)
    features = np.array(features)
    
    # set a Random Forest Regressor model
    regression = RandomForestRegressor(n_estimators=200,max_depth=10)
    lf = KFold(n_splits=5)
    splits = lf.split(features,targets,subs)
    
    # split the data and fit the model
    for i, (train_idx, test_idx) in enumerate(splits):
        X_train, y_train = features[train_idx], targets[train_idx]
        X_test, y_test = features[test_idx], targets[test_idx]
        regression.fit(X_train, y_train)
    
    return regression

def RunPulseRateAlgorithm(data_fl, ref_fl):
    """
    Compute the pulse rate algorithm and
    Returns errors and confidence
      """ 
    fs = 125
    win_len = 8
    win_shift = 2    
       
    targets, features, sigs, subs = PreprocessTestData(data_fl, ref_fl)

    error, confidence = [], []
    for i,feature in enumerate(features):
        # making a prediction
        est = reg.predict(np.reshape(feature, (1, -1)))[0]
        ppg, accx, accy, accz = sigs[i]        
        
        n = len(ppg) * 4
        freqs = np.fft.rfftfreq(n, 1/fs)
        fft = np.abs(np.fft.rfft(ppg,n))
        fft[freqs <= 40/60.0] = 0.0
        fft[freqs >= 240/60.0] = 0.0
    
        est_fs = est / 55.0
        fs_win = 30  / 60.0
        fs_win_e = (freqs >= est_fs - fs_win) & (freqs <= est_fs +fs_win)
        conf = np.sum(fft[fs_win_e])/np.sum(fft)
        
        error.append(np.abs((est-targets[i])))
        confidence.append(conf)
    return np.array(error), np.array(confidence)


In [4]:
result=Evaluate()
if result<15:
    print('Well done, {}'.format(result))
else:
    print('Try again, {}'.format(result))

Well done, 10.124341928517957


-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 

Run the cell above to check the performance of the algorithm with the training data. If the result is well done it means that the algorithm has an error below 15. 

> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.

We use the second channel of the PPG signal and three channels of the accelerometer corresponding to the axis x, y, and z. We could have used the mean of the two PPG channel instead.

> - **Algorithhm Description** will include the following:
>   - how the algorithm works

 The algorithm starts by preprocessing the data, then it makes a prediction of the pulse rate and calculates the errors and confidence.

>   - the specific aspects of the physiology that it takes advantage of

The PPG measures blood flow at the wrist. The LEDs in a PPG sensor shine a typically green light into your skin and your red blood cells absorb that green light. The reflected light is then measured by the photodetector. When your heart beats and blood perfuses through the wrist, there are more red blood cells that absorb the green light and the photodetector sees a smaller signal. As the heart fills back up with blood and blood leaves your wrist, the more green light is reflected back and the photodetector reading goes up. This oscillating waveform can be used to detect pulse rate. 

>   - a description of the algorithm outputs

The outputs of the algorithm are the confidence score of that prediction and the estimated frequency.

>   - caveats on algorithm outputs 

The area in which the confidence score is calculated is really small relatively to the total (conf = np.sum(fft[fs_win_e])/np.sum(fft))

>   - common failure modes

Movements of the fingers can produce high peaks that can alterate the outputs.

> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

We use a Random Forest Regressor with 200 estimators and max depht of 10 and we apply a Kfold with 5 splits. The metric that we use for calculating the performance was the mean absolute error.

-----
### Next Steps
You will now go to **Test Your Algorithm** (back in the Project Classroom) to apply a unit test to confirm that your algorithm met the success criteria. 